In [1]:
import urllib2
import json
import time
import threading
import timeit
import pandas as pd
from time import strftime,gmtime
import datetime

In [2]:
#web variables
dt = datetime.datetime.now()
UnixTime = int(time.mktime(dt.timetuple()))
url = 'https://query1.finance.yahoo.com/v8/finance/chart/AAPL?period1='+str(UnixTime-86400*30000)+'&period2='+str(UnixTime)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
       'Connection': 'keep-alive'}

In [3]:
#html data request 
try:
    request = urllib2.Request(url,headers = hdr)
    htmltext = urllib2.urlopen(request)
except:
    pass
data = json.load(htmltext)

In [4]:
#json unpack
timestamp = data['chart']['result'][0]['timestamp']
timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]

quote = data['chart']['result'][0]['indicators']['quote'][0]
adj_cls = data['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']
sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']

In [5]:
#pandas df create
stock_df = pd.DataFrame(quote)
stock_df.index = timestamp
stock_df['last_close'] = stock_df['close'].shift(1)
stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
stock_df['sma50']= sma50
stock_df['adj_close']= adj_cls
stock_df = stock_df.dropna()

#derived columns
stock_df['smaP'] = stock_df['close']/stock_df['sma50']
stock_df['volP'] = stock_df['volume']/stock_df['vol20']

stock_df['closeP'] = stock_df['close']/stock_df['last_close']
stock_df['highP'] = stock_df['high']/stock_df['last_close']
stock_df['lowP'] = stock_df['low']/stock_df['last_close']
stock_df['openP'] = stock_df['open']/stock_df['last_close']

In [6]:
stock_df

,close,high,low,open,volume,last_close,vol20,sma50,adj_close,smaP,volP,closeP,highP,lowP,openP
19810224,0.424107,0.428571,0.424107,0.428571,4244800.0,0.439732,5230960.0,0.5303,0.019295,0.799749,0.811476,0.964467,0.974619,0.964467,0.974619
19810225,0.450893,0.453125,0.450893,0.450893,4872000.0,0.424107,5178320.0,0.5291,0.020514,0.852188,0.940846,1.063158,1.068421,1.063158,1.063158
19810226,0.457589,0.459821,0.457589,0.457589,2710400.0,0.450893,4961880.0,0.5285,0.020819,0.865826,0.546245,1.014851,1.019802,1.014851,1.014851
19810227,0.473214,0.477679,0.473214,0.473214,3690400.0,0.457589,4597600.0,0.5289,0.021529,0.894714,0.802680,1.034146,1.043902,1.034146,1.034146
19810302,0.475446,0.477679,0.475446,0.475446,2940000.0,0.473214,4167240.0,0.5292,0.021631,0.898425,0.705503,1.004717,1.009434,1.004717,1.004717
19810303,0.468750,0.470982,0.468750,0.470982,4043200.0,0.475446,4072320.0,0.5291,0.021326,0.885938,0.992849,0.985915,0.990610,0.985915,0.990610
19810304,0.464286,0.466518,0.464286,0.466518,3427200.0,0.468750,4004280.0,0.5283,0.021123,0.878830,0.855884,0.990476,0.995238,0.990476,0.995238
19810305,0.462054,0.464286,0.462054,0.464286,1344000.0,0.464286,3723160.0,0.5269,0.021022,0.876928,0.360984,0.995192,1.000000,0.995192,1.000000
19810306,0.457589,0.462054,0.457589,0.462054,2900800.0,0.462054,3769080.0,0.5250,0.020819,0.871599,0.769631,0.990338,1.000000,0.990338,1.000000
19810309,0.421875,0.424107,0.421875,0.424107,3830400.0,0.457589,3787280.0,0.5219,0.019194,0.808345,1.011385,0.921951,0.926829,0.921951,0.926829
